<a href="https://colab.research.google.com/github/tg-bomze/collection-of-notebooks/blob/master/GPEN_DFL_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<b><font size="+4">GPEN for DeepFaceLab</font></b>

<b><font size="+2">Based on:</font></b>

**GitHub repository**: [GPEN](https://github.com/yangxy/GPEN)

Article: [GAN Prior Embedded Network for Blind Face Restoration in the Wild](https://arxiv.org/pdf/2105.06070.pdf)

Creator: **[yangxy](https://github.com/yangxy)**

This Сolab was created by [Bomze](https://github.com/tg-bomze/)

In [ ]:
#@title <b><font size="+3">0) Check GPU</font></b>
!nvidia-smi

In [ ]:
#@title <b><font size="+3">1) Prepare Colab machine</font></b>
!pip install ffmpeg-python
#!pip install torch==1.7.1 torchvision==0.8.2 torchaudio==0.7.2
#!pip install pip install opencv-python

from IPython.display import clear_output
from google.colab import files
!git clone https://github.com/yangxy/GPEN

!wget https://public-vigen-video.oss-cn-shanghai.aliyuncs.com/robin/models/RetinaFace-R50.pth && mv RetinaFace-R50.pth GPEN/weights/
!wget https://public-vigen-video.oss-cn-shanghai.aliyuncs.com/robin/models/GPEN-512.pth && mv GPEN-512.pth GPEN/weights/
#!wget https://public-vigen-video.oss-cn-shanghai.aliyuncs.com/robin/models/GPEN-1024-Color.pth && mv GPEN-1024-Color.pth GPEN/weights/

import os
import cv2
import ffmpeg
from PIL import Image
import numpy as np
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
%cd /content/GPEN

!wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip
!sudo unzip ninja-linux.zip -d /usr/local/bin/
!sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force
clear_output()

In [ ]:
#@title <b><font size="+3">2) Simple settings</font></b>
%%writefile /content/GPEN/main_file.py
import os
import cv2
import glob
import time
import numpy as np
from PIL import Image
from tqdm import tqdm
import __init_paths
from retinaface.retinaface_detection import RetinaFaceDetection
from face_model.face_gan import FaceGAN
from align_faces import warp_and_crop_face, get_reference_facial_points
from skimage import transform as tf

'''class FaceColorization(object):
    def __init__(self, base_dir='./', size=1024, model=None, channel_multiplier=2):
        self.facegan = FaceGAN(base_dir, size, model, channel_multiplier)
    # make sure the face image is well aligned. Please refer to face_enhancement.py
    def process(self, gray):
        return self.facegan.process(gray)#'''

class FaceEnhancement(object):
    def __init__(self, base_dir='./', size=512, model=None, channel_multiplier=2):
        self.facedetector = RetinaFaceDetection(base_dir)
        self.facegan = FaceGAN(base_dir, size, model, channel_multiplier)
        self.size = size
        self.threshold = 0.9

        # the mask for pasting restored faces back
        self.mask = np.zeros((512, 512), np.float32)
        cv2.rectangle(self.mask, (26, 26), (486, 486), (1, 1, 1), -1, cv2.LINE_AA)
        self.mask = cv2.GaussianBlur(self.mask, (101, 101), 11)
        self.mask = cv2.GaussianBlur(self.mask, (101, 101), 11)

        self.kernel = np.array((
                [0.0625, 0.125, 0.0625],
                [0.125, 0.25, 0.125],
                [0.0625, 0.125, 0.0625]), dtype="float32")

        # get the reference 5 landmarks position in the crop settings
        default_square = True
        inner_padding_factor = 0.25
        outer_padding = (0, 0)
        self.reference_5pts = get_reference_facial_points(
                (self.size, self.size), inner_padding_factor, outer_padding, default_square)

    def process(self, img):
        facebs, landms = self.facedetector.detect(img)
        
        orig_faces, enhanced_faces = [], []
        height, width = img.shape[:2]
        full_mask = np.zeros((height, width), dtype=np.float32)
        full_img = np.zeros(img.shape, dtype=np.uint8)

        for i, (faceb, facial5points) in enumerate(zip(facebs, landms)):
            if faceb[4]<self.threshold: continue
            fh, fw = (faceb[3]-faceb[1]), (faceb[2]-faceb[0])

            facial5points = np.reshape(facial5points, (2, 5))

            of, tfm_inv = warp_and_crop_face(img, facial5points, reference_pts=self.reference_5pts, crop_size=(self.size, self.size))
            
            # enhance the face
            ef = self.facegan.process(of)
            
            orig_faces.append(of)
            enhanced_faces.append(ef)
            
            tmp_mask = self.mask
            tmp_mask = cv2.resize(tmp_mask, ef.shape[:2])
            tmp_mask = cv2.warpAffine(tmp_mask, tfm_inv, (width, height), flags=3)

            if min(fh, fw)<100: # gaussian filter for small faces
                ef = cv2.filter2D(ef, -1, self.kernel)
            
            tmp_img = cv2.warpAffine(ef, tfm_inv, (width, height), flags=3)

            mask = tmp_mask - full_mask
            full_mask[np.where(mask>0)] = tmp_mask[np.where(mask>0)]
            full_img[np.where(mask>0)] = tmp_img[np.where(mask>0)]

        full_mask = full_mask[:, :, np.newaxis]
        img = cv2.convertScaleAbs(img*(1-full_mask) + full_img*full_mask)

        return img, orig_faces, enhanced_faces
        

if __name__=='__main__':
    indir = '/content/frames'
    outdir = '/content/result'
    scale = '2' #@param {type:"string"}
    #@markdown scale - how many times to enlarge the input image.
    os.makedirs(outdir, exist_ok=True)

    faceenhancer = FaceEnhancement(size=512, model='GPEN-512', channel_multiplier=2)
    #facecolorizer = FaceColorization(size=1024, model='GPEN-1024-Color', channel_multiplier=2)

    files = sorted(glob.glob(os.path.join(indir, '*.*g')))
    for n, file in tqdm(enumerate(files[:])):
        filename = os.path.basename(file)
        im = cv2.imread(file, cv2.IMREAD_COLOR)
        
        # Enchance
        if not isinstance(im, np.ndarray): print(filename, 'error'); continue
        im = cv2.resize(im, (0,0), fx=int(scale), fy=int(scale))
        im, orig_faces, enhanced_faces = faceenhancer.process(im)
        cv2.imwrite(os.path.join(outdir, '.'.join(filename.split('.')[:-1])+'.png'), im)#'''

        # Colorize
        '''colorf = facecolorizer.process(im)
        colorf = cv2.resize(colorf, (im.shape[1], im.shape[0]))
        cv2.imwrite(os.path.join(outdir, '.'.join(filename.split('.')[:-1])+'.png'), colorf)#'''

In [ ]:
#@title <b><font size="+3">3) Mount Google Drive</font></b>

from google.colab import drive
drive.mount('/content/drive')


In [ ]:
#@title <b><font size="+3">4) Enchance!</font></b>

import os

working_dir = "/content/drive/MyDrive/" #@param {type:"string"}
aligned_zip = True #@param {type:"boolean"}

#@markdown Pick a folder from your Google Drive.

#@markdown **working_dir** should contain **aligned** subfolder or **aligned.zip** file with JPG (or PNG) images that were extracted by DeepFaceLab *'faceset extract'* scripts.

#@markdown Results will be saved in **result** subfolder or **result.zip** file in **working_dir**.

#@markdown If working not in zip mode existing files will be skipped so you can easily continue in case of disconnect.

!rm -r '/content/frames' '/content/result'

if aligned_zip:
  zip_path = os.path.join(working_dir,'aligned.zip')
  if not os.path.isfile(zip_path):
    print('"{}" not found'.format(zip_path))
    sys.exit()
  !rsync -ah --progress "$zip_path" '/content/'
  !mkdir -p '/content/frames'
  !7z e '/content/aligned.zip' -o '/content/frames'
  !rm '/content/aligned.zip'
else:
  in_path = os.path.join(working_dir,'aligned')
  out_path = os.path.join(working_dir,'result')
  if not os.path.isdir(in_path):
    print('"{}" not found'.format(in_path))
    sys.exit()
  !rsync -ah --progress "$in_path" '/content'
  !mv /content/aligned /content/frames

!mkdir /content/result
clear_output()
!python main_file.py

if aligned_zip:
  !rm /content/result.zip
  !7z a /content/result.zip /content/result/*
  output_zip = os.path.join(working_dir,'result.zip')
  !rsync -ah --progress /content/result.zip "$output_zip"
  clear_output()
  print('Results saved to "{}"'.format(output_zip))
else:
  !rsync -ah --progress '/content/result' "$working_dir"
  clear_output()
  print('Results saved to "{}"'.format(out_path))